# 任务五 —— 从本地到云端——构建具备专业知识的AI学习助手
***

## 任务一 —— 本地“小脑” —— 运行本地开源模型
本题使用huggingface中的transformers库，加载轻量级模型到本地，并使用该模型回答了简单的问题。同时，为了直观的感受本地模型在进行推理时的资源使用情况，使用一个工具函数打印在推理前后cpu,gpu的使用情况。

### transformers库简介
Transformers库提供了大量训练好的模型的API和工具，我们可以在本地加载它们并用它们完成机器学习任务。同时，Transfomers库中加载的模型也能与Pytorch，tensorflow等深度学习框架兼容，这使得我们可以完成更多的操作。从Huggingface的官方文档中我们可以发现，此处使用的Qwen2模型只与Pytorch兼容。关于模型的信息，可以从Huggingface的网站上进行了解。

### 实现部分
在这里，我没有使用pipeline()完成该任务，因为我觉得它的使用太过简单，不能体现用模型执行推理的过程。我选择使用Autoclass加载模型，加载分词器，进行输入，得到输出。

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-1.5B-Instruct", dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

In [3]:
import os
import torch
import GPUtil
import psutil

# coplit写的工具函数，用于打印资源使用情况
def print_system_usage(tag=""):
    process = psutil.Process(os.getpid())
    cpu_percent = psutil.cpu_percent(interval=None)
    mem_info = process.memory_info().rss / 1024 / 1024  # MB
    print(f"\n[{tag}] CPU使用率: {cpu_percent:.1f}% | 内存占用: {mem_info:.1f} MB")

    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU {gpu.id}: 显存 {gpu.memoryUsed:.1f}/{gpu.memoryTotal:.1f} MB | 使用率 {gpu.load*100:.1f}%")

In [6]:
print_system_usage("推理前")
model_inputs = tokenizer(["请简要说明huggingface有哪些功能"], return_tensors="pt").to(model.device)
outputs = model.generate(**model_inputs, max_new_tokens=512, do_sample=False) #不采用随机采样，因为它会把我的问题进行补充
print_system_usage("推理后")
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]) #打印第一个生成结结果，虽然这里只有一个输入


[推理前] CPU使用率: 9.9% | 内存占用: 1223.3 MB
GPU 0: 显存 3200.0/8188.0 MB | 使用率 0.0%

[推理后] CPU使用率: 13.9% | 内存占用: 1223.3 MB
GPU 0: 显存 3200.0/8188.0 MB | 使用率 62.0%
请简要说明huggingface有哪些功能？ Hugging Face 是一个开源的机器学习和深度学习框架，它提供了许多强大的工具和库来帮助开发者进行自然语言处理、计算机视觉和其他领域的工作。以下是Hugging Face的一些主要功能：

1. **模型库**：Hugging Face 提供了一个庞大的模型库，包括预训练的模型（如 BERT、GPT 等）、自定义模型以及各种预训练任务的模型。

2. **代码生成器**：通过使用 Hugging Face 的代码生成器，可以快速构建新的模型或实现特定的任务。这使得开发人员能够专注于算法设计而非底层实现细节。

3. **API**：提供了一个 API 来访问和操作这些模型和数据集，使开发者能够轻松地集成到他们的项目中。

4. **社区支持**：Hugging Face 有一个活跃的社区，用户可以通过论坛、聊天室和问题跟踪系统与他人交流，分享知识和经验。

5. **文档和教程**：提供了详细的文档和教程，帮助开发者理解和使用 Hugging Face 的工具和服务。

6. **模型评估和优化**：提供了多种评估指标和优化策略，帮助开发者在训练过程中调整模型参数以提高性能。

7. **模型部署**：提供了用于将模型部署到服务器端的技术栈，例如 TensorFlow、PyTorch 和 PyTorch Hub。

8. **模型推理**：提供了用于从预训练模型中提取特征并进行推理的工具，适用于自然语言处理任务。

9. **模型管理**：提供了对模型的生命周期进行管理和监控的功能，包括模型版本控制、模型迁移等。

10. **模型共享**：允许开发者将自己的模型上传到 Hugging Face 的公共模型仓库，与其他开发者共享资源，共同推动模型的发展。

总之，Hugging Face 提供了一套全面且灵活的工具链，旨在帮助开发者更高效地进行自然语言处理、计算机视觉和其他领域的研究和应用。


### 结果记录与评价
在本地部署模型进行推理的过程中，GPU的使用率较高；该单元格的运行时长是19.1s，运行时长较长。就输出结果来说，由于我禁用了do_sample，所以答案中每一条句式较为固定。内容上，符合问题的要求。

***
## 任务二 —— 连接云端“超脑” —— 调用API

### 实现部分
本实现部分仿照官方文档中sdk的使用案例进行实现。为了与本地部署模型进行对比，我们使其max_new_tokens与本地部署实现中的值相等。

In [7]:
import os
import dashscope

In [8]:
messages = [{"role": "user", "content": "请简要说明huggingface有哪些功能"}]

In [9]:
response = dashscope.Generation.call(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    model="qwen3-235b-a22b-instruct-2507", 
    messages=messages,
    result_format='message',
    max_new_tokens = 512
    )

In [10]:
print(response["output"]["choices"][0]['message']['content'])  # 打印生成结果

Hugging Face 是一个专注于自然语言处理（NLP）和人工智能开源模型的平台，其主要功能包括：

1. **模型库（Model Hub）**  
   提供大量预训练模型（如 BERT、GPT、T5、RoBERTa 等），支持文本分类、生成、翻译、问答、命名实体识别等多种任务。用户可免费下载、使用或上传自己的模型。

2. **Transformers 库**  
   开源 Python 库，提供简单统一的接口调用各种预训练模型，兼容 PyTorch 和 TensorFlow，极大简化了模型的加载与微调流程。

3. **Datasets 库**  
   提供数千个公开数据集（如 SQuAD、GLUE、Common Crawl 等），支持一键加载和预处理，便于模型训练与评估。

4. **Tokenizers 库**  
   高性能分词工具，支持多种分词算法（如 BPE、WordPiece），可快速对文本进行编码，适配各类模型输入需求。

5. **Spaces（应用托管平台）**  
   允许用户部署机器学习演示应用（如聊天机器人、文本生成器），支持 Gradio 或 Streamlit 快速构建交互界面，实现模型可视化分享。

6. **AutoTrain 与推理 API**  
   - AutoTrain：自动化模型微调工具，适合无代码或低代码用户。
   - Inference API：提供云端模型推理服务，可直接通过 API 调用 Hugging Face 上的模型。

7. **协作与社区功能**  
   支持模型版本管理（Git-based）、评论、评分、Fork 和团队协作，形成活跃的开源 AI 社区。

8. **企业级解决方案（Hugging Face Enterprise）**  
   提供私有化部署、安全合规、定制化模型训练等服务，适用于企业客户。

总之，Hugging Face 构建了一个集模型共享、数据集管理、工具库、应用部署和社区交流于一体的完整 AI 生态系统，广泛应用于研究与工业领域。


### 结果分析与评价
这里的分析主要以与本地模型的对比相比较。首先时间上，云端的推理模块运行时间是16.7s，快于本地部署；其次，云端生成的答案句式丰富，排版整齐，语言更加凝练，故生成质量更高。

***
## 任务三 —— 框架赋能——使用LangChain构建专业知识库问答系统

### 数据预处理
在数据预处理部分，会按顺序进行`加载文档`，`文本切分`，`向量化`，`存储`四个过程。通过这四个过程，我们就可以将数据存入向量数据库，作为我们检索时的“知识库”使用

+ 文本加载

In [1]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
loader = DirectoryLoader(
    path="D:\\Studio_Recruitment\\task5\\",
    glob="**/*.txt",
    loader_cls=lambda file_path: TextLoader(file_path, encoding="utf-8")
    # 使用lambda函数，因为直接使用Textloader(encoding = 'utf-8')会直接生成实例从而不能传入file_path
)

docs = loader.load()
print(docs[0].page_content)  # 打印文档的内容

相对论（英语：Theory of relativity）是关于时空和引力的理论，主要由爱因斯坦创立，依其研究对象的不同可分为狭义相对论和广义相对论。相对论和量子力学的提出给物理学带来了革命性的变化，它们共同奠定了现代物理学的基础。相对论极大地改变了人类对宇宙和自然的“常识性”观念，提出了“同时的相对性”、“四维时空”、“弯曲时空”等全新的概念。不过近年来，人们对于物理理论的分类有了一种新的认识——以其理论是否是决定论的来划分经典与非经典的物理学，即“非经典的＝量子的”。在这个意义下，相对论仍然是一种经典的理论。
该理论取代了200年前主要由艾萨克·牛顿创立的力学理论，从而改变了20世纪的理论物理学和天文学，它引入的概念，包括时空、同时性之相对性、运动学、重力时间膨胀和洛伦兹收缩。在物理学领域，相对论改善了基本粒子的科学以及它们的基本相互作用，以及迎来核子时代。 另外，借由相对论，物理宇宙学及天体物理学预测了中子星、黑洞、引力波。传统上，在爱因斯坦提出相对论的初期，人们以所讨论的问题是否涉及非惯性参考系来作为狭义与广义相对论分类的标志。随着相对论理论的发展，这种分类方法越来越显出其缺点——参考系是跟观者有关的，以这样一个相对的物理对象来划分物理理论，被认为不能反映问题的本质。目前一般认为，狭义与广义相对论的区别在于所讨论的问题是否涉及引力（弯曲时空），即狭义相对论只涉及那些没有引力作用或者引力作用可以忽略的所有物理现象，而广义相对论则是讨论有引力作用时的物理学。用相对论的语言来说，就是狭义相对论的背景时空是平直的，即四维平凡流形配以闵氏度规，其曲率张量为零，又称闵氏时空；而广义相对论的背景时空则是弯曲的，其曲率张量不为零。



+ 文本切分

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap = 10,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    is_separator_regex=False
) # 定义文本切分器
texts = text_splitter.split_documents(docs) #切分文本
print(f"切分后的文本块数量: {len(texts)}")  # 打印切分后的文本块数量
print(texts[0].page_content)  # 打印第一个文本块的内容

切分后的文本块数量: 20
相对论（英语：Theory of


+ 向量化

In [14]:
%pip install --upgrade --quiet  langchain langchain-huggingface sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'})
docs_embeddings = embeddings.embed_documents([t.page_content for t in texts]) #计算文本块的嵌入表示
print(f"第一个文本块的嵌入表示维度: {len(docs_embeddings[0])}")  # 打印第一个文本块的嵌入表示维度
print(docs_embeddings[0][0:3])  # 打印第一个文本块的嵌

第一个文本块的嵌入表示维度: 512
[-0.008902044035494328, -0.026875978335738182, -0.03574147820472717]


+ 存储

In [4]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs,embeddings)
vectorstore.save_local("faiss_index")  # 保存FAISS索引到本地目录
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

+ 加载模型

In [5]:
from langchain_community.chat_models.tongyi import ChatTongyi

chat_llm = ChatTongyi(model="qwen3-235b-a22b-instruct-2507", streaming=False)

+ 构建RAG

In [14]:
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate

# Zero-shot CoT Prompt
prompt = ChatPromptTemplate.from_template("""
你是一个科学专家，请基于以下检索到的资料回答问题。
如果资料不足，请回答“我不知道”，不要编造。

资料：
{context}

问题：
{question}

请一步一步地推理(Let's think step by step).
在“思考过程”部分必须写出清晰的多步推理(至少3步),
最后在“最终答案”部分给出简洁结论。

输出格式：
思考过程：
1. ...
2. ...
3. ...
最终答案：...
""")


qa_chain = RetrievalQA.from_chain_type(
    llm=chat_llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type="stuff",   # ✅ 合法的 chain_type
    chain_type_kwargs={"prompt": prompt},
    input_key="question",   # 输入键与 prompt 占位符一致
    output_key="answer"     # 输出键自定义为 answer
)

query = "请说明相对论包括哪两种"
result = qa_chain({"question": query})
print("回答:", result["answer"])


回答: 思考过程：  
1. 根据提供的资料，相对论是关于时空和引力的理论，主要由爱因斯坦创立。这一理论并不是单一的整体，而是可以根据研究对象的不同进行分类。  
2. 资料中明确指出：“依其研究对象的不同可分为狭义相对论和广义相对论。”这说明相对论主要分为两种类型，其区分依据是所研究的问题是否涉及引力。  
3. 进一步解释，狭义相对论适用于没有引力或引力可以忽略的情况，其时空背景是平直的闵氏时空；而广义相对论则将引力纳入考虑，描述的是弯曲的时空结构，适用于有引力作用的物理现象。

最终答案：相对论包括狭义相对论和广义相对论两种。


### 遇到的问题及解决：
1. 任务二时打印答案的格式问题。解决：根据结构得到正确的访问方式
2. 构建RAG时找不到官方教程的问题，不知道怎么使用相关函数和方法。解决：理清楚构造的流程，问AI相关函数和方法，并结合具体案例学习使用方法。
3. 使用zero-shot CoT时，输出没展示思考过程。解决：限定输出格式。

***
## 写在最后
+ 任务完成情况：任务一二基本都弄明白了，任务三在构建RAG时还只能依葫芦画瓢。
+ 最后还差根据提交格式改进的部分，留到国庆吧，因为还有两题，但只有三周了。